## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Marsh Harbour,BS,2022-04-01 01:23:41,26.5412,-77.0636,Clouds,76.26,75,60,18.39
1,1,Upernavik,GL,2022-04-01 01:23:41,72.7868,-56.1549,Clouds,11.05,97,100,9.40
2,2,Chuy,UY,2022-04-01 01:23:41,-33.6971,-53.4616,Clouds,55.99,77,86,5.84
3,3,Punta Arenas,CL,2022-04-01 01:23:41,-53.1500,-70.9167,Clouds,50.11,62,75,24.16
4,4,Ushuaia,AR,2022-04-01 01:19:32,-54.8000,-68.3000,Clouds,49.66,57,20,11.50


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Current Description,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Marsh Harbour,BS,2022-04-01 01:23:41,26.5412,-77.0636,Clouds,76.26,75,60,18.39
6,6,Galle,LK,2022-04-01 01:23:42,6.0367,80.2170,Clouds,77.99,82,20,1.77
9,9,Puerto Ayora,EC,2022-04-01 01:23:43,-0.7393,-90.3518,Clouds,73.54,100,100,1.95
11,11,Atuona,PF,2022-04-01 01:23:43,-9.8000,-139.0333,Clouds,78.84,75,79,8.23
15,15,Sabha,LY,2022-04-01 01:23:44,27.0377,14.4283,Clouds,73.89,18,100,10.49
17,17,Caravelas,BR,2022-04-01 01:23:45,-17.7125,-39.2481,Clear,76.57,87,4,5.53
22,22,Port Augusta,AU,2022-04-01 01:14:30,-32.5000,137.7667,Clear,70.34,40,0,12.30
27,27,Hilo,US,2022-04-01 01:23:32,19.7297,-155.0900,Rain,78.24,84,100,6.91
33,33,Mount Gambier,AU,2022-04-01 01:23:48,-37.8333,140.7667,Clear,65.32,44,0,11.92
39,39,Arlit,NE,2022-04-01 01:23:50,18.7369,7.3853,Clear,78.22,4,3,7.34


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                207
City                   207
Country                206
Date                   207
Lat                    207
Lng                    207
Current Description    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df =  preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", 'Current Description', "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = " "
hotel_df.head(10)

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
0,Marsh Harbour,BS,Clouds,76.26,26.5412,-77.0636,
6,Galle,LK,Clouds,77.99,6.0367,80.2170,
9,Puerto Ayora,EC,Clouds,73.54,-0.7393,-90.3518,
11,Atuona,PF,Clouds,78.84,-9.8000,-139.0333,
15,Sabha,LY,Clouds,73.89,27.0377,14.4283,
17,Caravelas,BR,Clear,76.57,-17.7125,-39.2481,
22,Port Augusta,AU,Clear,70.34,-32.5000,137.7667,
27,Hilo,US,Rain,78.24,19.7297,-155.0900,
33,Mount Gambier,AU,Clear,65.32,-37.8333,140.7667,
39,Arlit,NE,Clear,78.22,18.7369,7.3853,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:    
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print('Hotel not found... skipping.')
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(layout={
        'width': '800px',
        'height': '800px',
        'padding': '3px',
        'border': '1px solid black'},
        center=(0, 0), zoom_level=1.7)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='3px', width='800px'))